### Set up notebook

In [1]:
import os
os.chdir('../')
os.getcwd()

'/Users/Cisco/Desktop/Master_Dauphine/M2_IEF/cours/semestre2/MachineLearning/projet'

### Import raw data

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('data/cac40_v3.csv').drop('Unnamed: 0', axis=1)
data

,TICKER,annee,mois,jour,OP,UP,DO,CL,VO,RDMT_J,...,strategique,transaction,actionnaires,coentreprise,construction,collaboration,developpement,l_acquisition,participation,recommandation
0,AC,2007,10,1,50.820,52.450,50.720,52.440,939518,0.045576,...,0,0,0,0,0,0,0,0,0,0
1,AC,2007,10,10,57.600,57.900,56.480,56.740,1167710,0.005640,...,0,0,0,0,0,0,0,0,0,0
2,AC,2007,10,11,56.570,57.900,56.570,57.060,910369,-0.005258,...,0,0,0,0,0,0,0,0,0,0
3,AC,2007,10,12,57.100,57.100,55.500,56.760,996349,-0.011628,...,0,0,0,0,0,0,0,0,0,0
4,AC,2007,10,15,56.760,57.290,55.770,56.100,573647,-0.003209,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106537,VIV,2017,9,4,20.065,20.235,20.030,20.100,2250835,0.002736,...,0,0,0,0,0,0,0,0,0,0
106538,VIV,2017,9,5,20.095,20.290,19.985,20.155,3095238,-0.001985,...,0,0,0,0,0,0,0,0,0,0
106539,VIV,2017,9,6,20.080,20.210,20.015,20.115,2181925,-0.006463,...,0,0,0,0,0,0,0,0,0,0
106540,VIV,2017,9,7,20.185,20.185,19.920,19.985,4038684,-0.002252,...,0,0,0,0,0,0,0,0,0,0


### Question 1)

Donner les charactéristiques de la base :
- Nombre de lignes
- Nombre de colonnes
- Format / Type de colonnes

In [4]:
print(f"Nombre de lignes : {data.shape[0]}")
print(f"Nombre de colonees : {data.shape[1]}")

Nombre de lignes : 106542
Nombre de colonees : 280


In [ ]:
for series_name, series in data.iteritems():
    print(f"Colonne : {series_name}, Type : {series.dtype}")

### Question 2)

Présenter quelques statistiques de la base

In [6]:
data.describe()

,annee,mois,jour,OP,UP,DO,CL,VO,RDMT_J,RDMT_S,...,strategique,transaction,actionnaires,coentreprise,construction,collaboration,developpement,l_acquisition,participation,recommandation
count,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,1.065420e+05,106542.000000,106542.000000,...,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000,106542.000000
mean,2012.036887,6.538060,15.778134,52.751368,53.330280,52.129056,52.738273,2.697387e+06,0.000312,0.001493,...,0.000920,0.000826,0.000676,0.000516,0.000404,0.000300,0.000845,0.001323,0.001577,0.001577
std,3.088057,3.373755,8.739708,41.933400,42.278984,41.565587,41.932864,3.253353e+06,0.021855,0.047802,...,0.030315,0.028728,0.025987,0.022715,0.020086,0.017328,0.029052,0.036355,0.039678,0.039678
min,2007.000000,1.000000,1.000000,2.240000,2.274000,2.120000,2.196000,0.000000e+00,-0.262421,-0.375313,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2009.000000,4.000000,8.000000,23.700000,24.035000,23.315000,23.695000,6.933412e+05,-0.010156,-0.022804,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2012.000000,7.000000,16.000000,42.095000,42.600000,41.527500,42.065000,1.549323e+06,0.000174,0.001782,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2015.000000,9.000000,23.000000,67.273375,67.990000,66.550000,67.250000,3.488776e+06,0.010599,0.025577,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2017.000000,12.000000,31.000000,394.950000,397.000000,391.050000,395.350000,6.382521e+07,0.263158,0.520471,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Question 3)

Montrer que, _quelque soit le ticker_, la liste des mots apparaissant sur 400 lignes et dont le rendement mensuel est en moyenne supérieur à 1% est telle que la table dans le sujet.  

In [7]:
import numpy as np

word_flag = "avec"
ix_word_flag = np.argmax(data.columns == word_flag)
all_words = data.columns[ix_word_flag:]
not_words = data.columns[:ix_word_flag]

In [8]:
# On veut l'effet du mot, indépendamment de ticker
freq_word_ret = {}
for word in all_words:
    if data[word].sum() > 400:
        mean_return = np.mean(data["RDMT_M"].loc[data[word] == 1])
        if mean_return > 0.01:
            freq_word_ret[word] = mean_return

In [9]:
freq_word_ret

{'part': 0.011925091555993018,
 'plus': 0.010119257588012652,
 'pour': 0.013645055868837895,
 'euros': 0.010361023597468611,
 'group': 0.012940358066579583,
 'passe': 0.01376329128793489,
 'titre': 0.012178265842055443,
 'groupe': 0.014646081429147735,
 'nouvel': 0.011500023035921798,
 'releve': 0.010296593113268828,
 'actions': 0.01016788292303123,
 'capital': 0.011974752777913301,
 'contrat': 0.01057704828737451,
 'nouveau': 0.011895224332522554,
 'capital.': 0.011040725408502755,
 'nouvelle': 0.011975116627910317,
 'objectif': 0.014702096122149608,
 'resultat': 0.011162225879723832}

<b>Filtrer la base sur l'apparition de ces mots</b>

In [37]:
keepers = list(freq_word_ret.keys())
data["KEEP_ROW"] = data[keepers].sum(axis=1)
data = data.loc[~(data["KEEP_ROW"] == 0)].drop("KEEP_ROW", axis=1).reset_index()

In [38]:
data

,index,TICKER,annee,mois,jour,OP,UP,DO,CL,VO,...,strategique,transaction,actionnaires,coentreprise,construction,collaboration,developpement,l_acquisition,participation,recommandation
0,744,AC,2010,10,11,27.370,27.655,27.165,27.585,583747,...,0,0,0,0,0,0,0,0,0,0
1,757,AC,2010,10,28,29.220,29.455,28.950,29.130,752703,...,0,0,0,0,0,0,0,0,0,0
2,761,AC,2010,10,6,26.950,27.955,26.950,27.670,1747197,...,0,0,0,0,0,0,0,0,0,0
3,778,AC,2010,11,26,33.415,33.775,33.075,33.660,871648,...,0,0,0,0,0,0,0,0,0,0
4,780,AC,2010,11,3,29.745,29.980,29.580,29.830,880633,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7124,106529,VIV,2017,9,20,21.075,21.190,21.010,21.020,2787945,...,0,0,0,0,0,0,0,0,0,0
7125,106532,VIV,2017,9,25,20.710,20.875,20.685,20.755,2082640,...,0,0,0,0,0,0,0,0,0,0
7126,106533,VIV,2017,9,26,20.765,20.865,20.695,20.725,1622952,...,0,0,0,0,0,0,0,0,0,0
7127,106536,VIV,2017,9,29,21.060,21.420,20.890,21.420,6358735,...,0,0,0,0,0,0,0,0,0,0
